In [5]:
from transformers import pipeline, set_seed, logging
set_seed(42)
logging.set_verbosity_error()

In [2]:
# Setting up the underlying and primary models
task = "question-answering"
models = {'gpt2':           pipeline(task, model="openai-community/gpt2"),
          'roberta':     pipeline(task, model="deepset/roberta-base-squad2"),
          'tiny_bert':        pipeline(task, model="Intel/dynamic_tinybert"),
          'amzn_qanlu':    pipeline(task, model="AmazonScience/qanlu"),
          }

primary_model = 'tiny_bert'


In [3]:
question = "Who won the superbowl in 2024?"
context = "The San Fransico 49ers took an early lead but Kansas outperformed them at overtime in SuperBowl 2024."
prompt = {'context': context, 'question': question}

text_generation_params = {'max_length':100, 'truncation':True}

answers = dict()
for model_name, model in models.items():
    answer = model(prompt, **text_generation_params)
    answer = answer #Keeping only the top response for now
    answers[model_name] = answer
answers

{'gpt2': {'score': 0.02567078173160553,
  'start': 41,
  'end': 95,
  'answer': ' but Kansas outperformed them at overtime in SuperBowl'},
 'roberta': {'score': 0.8439465761184692,
  'start': 46,
  'end': 52,
  'answer': 'Kansas'},
 'tiny_bert': {'score': 0.5538791418075562,
  'start': 46,
  'end': 52,
  'answer': 'Kansas'},
 'amzn_qanlu': {'score': 0.9204440116882324,
  'start': 46,
  'end': 52,
  'answer': 'Kansas'}}

In [4]:

ensemble_prompt = {'context': context + '\n' + str(answers.values()),
                   'question': question
                   }

ensemble_params = {'max_length':None, 'truncation':True}
answer = models[primary_model](ensemble_prompt, **ensemble_params)
answer

{'score': 0.47646719217300415, 'start': 46, 'end': 52, 'answer': 'Kansas'}